In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import re

In [3]:
# Load PhoBERT tokenizer and model (bert-base version)
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [11]:
def preprocess_text(text):
    # Loại bỏ các ký tự đặc biệt bằng regular expressions
    processed_text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ tất cả các ký tự không phải chữ, số và khoảng trắng
    # Loại bỏ dấu thừa và khoảng trắng không cần thiết
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()  # Xóa khoảng trắng thừa

    return processed_text.lower()

In [9]:
def text_to_vector(text):
    tokens = preprocess_text(text)
    inputs = phobert_tokenizer(
        " ".join(tokens),
        return_tensors="pt",
        truncation=True,
        max_length=256
    )
    outputs = phobert_model(**inputs)
    vector = outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()
    return vector

In [4]:
path = '/content/drive/MyDrive/DS_KLTN/vector/'

In [14]:
# Load the CSV file
product_name = pd.read_csv(path + 'name_description_highligh.csv')[['product_id', 'name']]

In [15]:
product_name.head()

,product_id,name
0,99958075,Quần đùi short gió nam thể thao Basic trẻ...
1,186133766,"Quần Short Kaki Nam US ARMY, Quần Túi Hộp Nam ..."
2,214666377,"Áo thun polo, thời trang nam họa tiết thêu PAC..."
3,56614642,"Combo 5 quần sịp boxer chất cotton mềm mịn, th..."
4,172210350,"Hộp 4 quần boxer nam thun lạnh, quần lót nam c..."


In [16]:
# Process each category and convert it to a vector
vectors = []
ids = []
for index, row in product_name.iterrows():
    product_id = row['product_id']
    product_name = row['name']

    # Convert category name to vector
    vector = text_to_vector(product_name)

    # Append the category_id and vector
    ids.append(product_id)
    vectors.append(vector)

In [17]:
# Save the vectors and ids as a .npy file
vector_file_path = 'prodcut_name_vectors.npy'
np.save(path + vector_file_path, {"id": ids, "vector": vectors})

In [18]:
# Load the vectors and IDs from the .npy file
data = np.load(path + vector_file_path, allow_pickle=True).item()

In [19]:
# Extract the ids and vectors
ids = data['id']
vectors = data['vector']

In [ ]:
vectors